## Creating positive track list of your favourite artist using SpotifyAPI
inpired by http://rcharlie.com/2017-02-16-fitteR-happieR/

https://developer.spotify.com/web-api/


In [1]:
artist_name='Radiohead'

In [2]:
import requests
req= requests.get(url='https://api.spotify.com/v1/search', params= {'q':artist_name, 'type':'artist'}
                 )
resp=req.json()

In [3]:
import pprint as pp
pp.pprint(resp)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Radiohead&type=artist&offset=0&limit=20',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
                        'followers': {'href': None, 'total': 2049579},
                        'genres': ['alternative rock',
                                   'indie rock',
                                   'melancholia',
                                   'permanent wave',
                                   'rock'],
                        'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
                        'id': '4Z8W4fKeB5YxbusRsdQVPb',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/afcd616e1ef2d2786f47b3b4a8a6aeea24a72adc',
                                    'width': 640},
                                   {'height': 320,
                                    'url': 'https

In [4]:
## parse json response and combine needed artist info (name and URI) into dataframe
import pandas as pd
from pandas.io.json import json_normalize
artists_df=json_normalize(resp['artists']['items'])
artists_df=artists_df[['name', 'uri']]
artists_df['uri']= artists_df['uri'].apply(lambda x : x.replace('spotify:artist:', ''))
artists_df.head()

,name,uri
0,Radiohead,4Z8W4fKeB5YxbusRsdQVPb
1,Radiohead Tribute Band,0ADkBHZhR2cVfANgK5gHQO


In [5]:
'https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artists_df.loc[0, 'uri'])

'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums'

In [6]:
##using artists URI get artists albums and songs 
req= requests.get(url='https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artists_df.loc[0, 'uri']),
                 params= {'limit':50})
resp_albums=req.json()
pp.pprint(resp_albums)


{'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums?offset=0&limit=50&album_type=single,album,compilation,appears_on,ep',
 'items': [{'album_type': 'album',
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
                         'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
                         'id': '4Z8W4fKeB5YxbusRsdQVPb',
                         'name': 'Radiohead',
                         'type': 'artist',
                         'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
            'available_markets': ['AD',
                                  'AR',
                                  'AT',
                                  'AU',
                                  'BE',
                                  'BG',
                                  'BO',
                                  'BR',
                                  'CA',
                                

In [7]:
albums_df=json_normalize(resp_albums['items'])
albums_df["artist_name"]=albums_df.artists.apply(lambda x: x[0]["name"])
albums_df=albums_df[['id','type', 'name', "artist_name"]]
albums_df.head(20)

,id,type,name,artist_name
0,6vuykQgDLUCiZ7YggIpLM9,album,A Moon Shaped Pool,Radiohead
1,47xaqCsJcYFWqD1gwujl1T,album,TKOL RMX 1234567,Radiohead
2,1DBkJIEoeHrTX4WCBQGcCi,album,The King Of Limbs,Radiohead
3,7eyQXxuf2nGj9d2367Gi5f,album,In Rainbows,Radiohead
4,36lJLPoPPOKNFddTAcirnc,album,In Rainbows Disk 2,Radiohead
5,6Eo5EkmdLvZrONzi046iC2,album,Com Lag: 2+2=5,Radiohead
6,1oW3v5Har9mvXnGk0x4fHm,album,Hail To the Thief,Radiohead
7,6svTt5o2lUgIrgYDKVmdnD,album,I Might Be Wrong,Radiohead
8,6V9YnBmFjWmXCBaUVRCVXP,album,Amnesiac,Radiohead
9,19RUXBFyM4PpmrLRdtqWbp,album,Kid A,Radiohead


In [8]:
albums_df.artist_name.unique()


array(['Radiohead', 'Various Artists', '矢沢朋子', 'Will Bates', 'Lvtum'], dtype=object)

In [9]:
albums_df.type.unique()

array(['album'], dtype=object)

In [10]:
albums_df=albums_df[ (albums_df['artist_name']==artist_name) & (albums_df['type']=='album')]
#albums_df=albums']_df[ albums_df.artist_name=="Radiohead"]
albums_df.artist_name.unique()
albums_df.shape

(32, 4)

### Pagination

In [11]:
## we need to support pagination
def get_artist_albums(artist_id):
    columns=['id','type', 'name' ]
    albums_df=pd.DataFrame()
    limit=50 #set the number of albums to retrieve from API, 50 is the max
    offset=0 #The index of the first album to return. Default: 0 (i.e., the first album)
    total=limit
    while total> offset:
        req= requests.get(url='https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artist_id),
                 params= {'limit':limit, 'offset':offset})
        resp_albums=req.json()
        #pp.pprint(json_normalize(resp_albums['items'])[columns],)
        albums_df=albums_df.append(json_normalize(resp_albums['items'])[columns], ignore_index=True)
        #pp.pprint(albums_df)
        ##read the total from response and update offset
        total=resp_albums['total']
        offset=offset + limit 
        print ('TOTAL: %d, OFFSET: %d' % (total, offset))
    return albums_df 

all_albums=get_artist_albums(artists_df.loc[0, 'uri'])
print (all_albums.shape)
all_albums.head(100)


TOTAL: 95, OFFSET: 50
TOTAL: 95, OFFSET: 100
(95, 3)


,id,type,name
0,6vuykQgDLUCiZ7YggIpLM9,album,A Moon Shaped Pool
1,47xaqCsJcYFWqD1gwujl1T,album,TKOL RMX 1234567
2,1DBkJIEoeHrTX4WCBQGcCi,album,The King Of Limbs
3,7eyQXxuf2nGj9d2367Gi5f,album,In Rainbows
4,36lJLPoPPOKNFddTAcirnc,album,In Rainbows Disk 2
5,6Eo5EkmdLvZrONzi046iC2,album,Com Lag: 2+2=5
6,1oW3v5Har9mvXnGk0x4fHm,album,Hail To the Thief
7,6svTt5o2lUgIrgYDKVmdnD,album,I Might Be Wrong
8,6V9YnBmFjWmXCBaUVRCVXP,album,Amnesiac
9,19RUXBFyM4PpmrLRdtqWbp,album,Kid A


In [12]:
##Having all the album ids we can pull the full track list for each album
def get_tracks(album_id):
    req_tracks=requests.get(url='https://api.spotify.com/v1/albums/{id}/tracks'.format(id=album_id), 
                           params={'limit':50}) ##here make an assumption that track list is <=50 songs
    json_resp=req_tracks.json()
    return json_resp['items']


all_tracks= list(map(get_tracks,albums_df.id))


In [13]:
tracks_df=json_normalize(sum(all_tracks,[]))
tracks_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls.spotify,href,id,name,preview_url,track_number,type,uri
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",1,220609,False,https://open.spotify.com/track/3pcCifdPTc2Bbqm...,https://api.spotify.com/v1/tracks/3pcCifdPTc2B...,3pcCifdPTc2BbqmWpEhtUd,Burn the Witch,https://p.scdn.co/mp3-preview/c5b0b198ee3f0459...,1,track,spotify:track:3pcCifdPTc2BbqmWpEhtUd
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",1,384438,False,https://open.spotify.com/track/1uRxyAup7OYrlh2...,https://api.spotify.com/v1/tracks/1uRxyAup7OYr...,1uRxyAup7OYrlh2SHJb80N,Daydreaming,https://p.scdn.co/mp3-preview/fbf970f6ee1e7d84...,2,track,spotify:track:1uRxyAup7OYrlh2SHJb80N
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",1,281011,False,https://open.spotify.com/track/5rIhBK9aaVMck0W...,https://api.spotify.com/v1/tracks/5rIhBK9aaVMc...,5rIhBK9aaVMck0W2YtOwci,Decks Dark,https://p.scdn.co/mp3-preview/3500e1883855236f...,3,track,spotify:track:5rIhBK9aaVMck0W2YtOwci
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",1,224698,False,https://open.spotify.com/track/1kBGeOp1CDUHVdb...,https://api.spotify.com/v1/tracks/1kBGeOp1CDUH...,1kBGeOp1CDUHVdbK4ergqo,Desert Island Disk,https://p.scdn.co/mp3-preview/0a484b7a356f34f1...,4,track,spotify:track:1kBGeOp1CDUHVdbK4ergqo
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",1,367348,False,https://open.spotify.com/track/4CzTgOmc3Sdm4Eg...,https://api.spotify.com/v1/tracks/4CzTgOmc3Sdm...,4CzTgOmc3Sdm4EgKQWzjQl,Ful Stop,https://p.scdn.co/mp3-preview/7b544b63c8c8aa64...,5,track,spotify:track:4CzTgOmc3Sdm4EgKQWzjQl


In [14]:
to_drop_cols=['available_markets', 'disc_number', 'explicit', 'external_urls.spotify', 'type', 'track_number', 'uri', 'preview_url']

In [15]:
tracks_df=tracks_df.drop(to_drop_cols, axis=1)

In [16]:
tracks_df=tracks_df.drop_duplicates(['id'])

In [17]:
tracks_df.shape
tracks_df["artist_name"]=tracks_df.artists.apply(lambda x: x[0]["name"])
tracks_df=tracks_df.drop(["artists"], axis=1)

In [18]:
tracks_df[tracks_df.artist_name!="Radiohead"].head(20)

,duration_ms,href,id,name,artist_name


In [19]:
tracks_df.head()

,duration_ms,href,id,name,artist_name
0,220609,https://api.spotify.com/v1/tracks/3pcCifdPTc2B...,3pcCifdPTc2BbqmWpEhtUd,Burn the Witch,Radiohead
1,384438,https://api.spotify.com/v1/tracks/1uRxyAup7OYr...,1uRxyAup7OYrlh2SHJb80N,Daydreaming,Radiohead
2,281011,https://api.spotify.com/v1/tracks/5rIhBK9aaVMc...,5rIhBK9aaVMck0W2YtOwci,Decks Dark,Radiohead
3,224698,https://api.spotify.com/v1/tracks/1kBGeOp1CDUH...,1kBGeOp1CDUHVdbK4ergqo,Desert Island Disk,Radiohead
4,367348,https://api.spotify.com/v1/tracks/4CzTgOmc3Sdm...,4CzTgOmc3Sdm4EgKQWzjQl,Ful Stop,Radiohead


### Get Audio Features for a Track
Authorization Required. A valid access token from the Spotify Accounts service: see the Web API Authorization Guide for details:
https://developer.spotify.com/web-api/authorization-guide/

Create spotify Developers account (you need to be registered Spotify user) https://developer.spotify.com/my-applications/#!/applications 

In [20]:
##request authentication token using your Client Id and Client Secret
import config
#Authorization Required. Base 64 encoded string that contains the client ID and client secret key. 
#The field must have the format: Authorization: Basic <base64 encoded client_id:client_secret>
from requests.auth import HTTPBasicAuth

auth= requests.post('https://accounts.spotify.com/api/token',data={'grant_type':'client_credentials'},
                         auth=HTTPBasicAuth(config.SpotifyClient.client_id, config.SpotifyClient.client_secret))

##read the status code - 200 means success
auth.status_code

200

In [21]:
##using access token you can now autorize and get audio features for a track 
auth_token=auth.json()['access_token']
track_audio_features= list(map(lambda x: requests.get('https://api.spotify.com/v1/audio-features/{track_id}'.format(track_id=x), 
                                        headers={"Authorization": "Bearer %s" %auth_token}).json(), tracks_df.id ))
 


In [22]:
audiof_df=json_normalize(track_audio_features)

In [23]:
audiof_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.303,https://api.spotify.com/v1/audio-analysis/3pcC...,0.541,220609,0.847,3pcCifdPTc2BbqmWpEhtUd,0.271,11,0.109,-6.520,1,0.0297,148.937,4,https://api.spotify.com/v1/tracks/3pcCifdPTc2B...,audio_features,spotify:track:3pcCifdPTc2BbqmWpEhtUd,0.625
1,0.968,https://api.spotify.com/v1/audio-analysis/1uRx...,0.299,384439,0.263,1uRxyAup7OYrlh2SHJb80N,0.857,9,0.126,-13.207,0,0.0323,137.848,3,https://api.spotify.com/v1/tracks/1uRxyAup7OYr...,audio_features,spotify:track:1uRxyAup7OYrlh2SHJb80N,0.115
2,0.668,https://api.spotify.com/v1/audio-analysis/5rIh...,0.560,281011,0.500,5rIhBK9aaVMck0W2YtOwci,0.833,2,0.117,-10.829,0,0.0272,139.174,4,https://api.spotify.com/v1/tracks/5rIhBK9aaVMc...,audio_features,spotify:track:5rIhBK9aaVMck0W2YtOwci,0.258
3,0.868,https://api.spotify.com/v1/audio-analysis/1kBG...,0.561,224698,0.282,1kBGeOp1CDUHVdbK4ergqo,0.529,0,0.110,-13.241,1,0.0286,112.011,1,https://api.spotify.com/v1/tracks/1kBGeOp1CDUH...,audio_features,spotify:track:1kBGeOp1CDUHVdbK4ergqo,0.153
4,0.353,https://api.spotify.com/v1/audio-analysis/4CzT...,0.426,367349,0.661,4CzTgOmc3Sdm4EgKQWzjQl,0.940,0,0.619,-9.877,1,0.0503,151.191,3,https://api.spotify.com/v1/tracks/4CzTgOmc3Sdm...,audio_features,spotify:track:4CzTgOmc3Sdm4EgKQWzjQl,0.179


In [24]:
audiof_df.shape

(211, 18)

### Quantyfying positiveness of a song - easy approach
valence	-	A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

In [25]:
tracks_sorted=audiof_df[["uri", "valence"]].sort_values(by=["valence"], axis=0, ascending=False, inplace=False)
tracks_sorted

,uri,valence
38,spotify:track:6dsq7Nt5mIFzvm5kIYNORy,0.8480
13,spotify:track:64lecUR19lBSu317AzVZv3,0.8370
46,spotify:track:15ea10YpJIl3mJq75yzqsD,0.8180
153,spotify:track:2O6h9bcoyqbg9OYTqHJ535,0.7880
158,spotify:track:7oC7bUp0pp4IOPlVTKpqVU,0.7630
65,spotify:track:4blz5SBUxKbtDNwMWstGNG,0.7630
33,spotify:track:0eECFDnWy0RdjMmJ8NOeAL,0.7510
116,spotify:track:53aFGAl0Vebwp2bo8wtdWR,0.7280
74,spotify:track:77SPrxY5gn6VPrKPGrfLnM,0.7260
69,spotify:track:6CVdTDYoDbwYj4xn8u5Gha,0.7240


### Creating a playlist
Now let's create a playlist of 20 most positive songs.To do this we need to:
1. create an empty playlist by making a POST request to playlists endpoint
    https://developer.spotify.com/web-api/create-playlist/
2. add tracks to created playlist
    https://developer.spotify.com/web-api/add-tracks-to-playlist/
    

Your application requests authorization
GET https://accounts.spotify.com/authorize

In [26]:
import six
import six.moves.urllib.parse as urllibparse
OAUTH_AUTHORIZE_URL = 'https://accounts.spotify.com/authorize'
OAUTH_TOKEN_URL = 'https://accounts.spotify.com/api/token'
scope='playlist-modify-public playlist-modify-private'
def _normalize_scope(scope):
    scopes = scope.split()
    print(scopes)
    scopes.sort()
    return ' '.join(scopes)
print (_normalize_scope(scope))

['playlist-modify-public', 'playlist-modify-private']
playlist-modify-private playlist-modify-public


In [27]:

params={
    'client_id':config.SpotifyClient.client_id,
    'redirect_uri':config.SpotifyClient.redirect_uri,
    'scope':_normalize_scope(scope),
    'response_type':'code'
}
urlparams = urllibparse.urlencode(params)
urlparams
#auth=requests.get('https://accounts.spotify.com/authorize', urlparams)
oauth_url=("%s?%s" % (OAUTH_AUTHORIZE_URL, urlparams))

oauth_url



['playlist-modify-public', 'playlist-modify-private']


'https://accounts.spotify.com/authorize?client_id=a41c83dec385446791496f4cd09c5593&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=playlist-modify-private+playlist-modify-public&response_type=code'

In [28]:
import webbrowser
try:
    webbrowser.open(oauth_url)
    print("Opened %s in your browser" % oauth_url)
except:
    print("Please navigate here: %s" % oauth_url)



Opened https://accounts.spotify.com/authorize?client_id=a41c83dec385446791496f4cd09c5593&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=playlist-modify-private+playlist-modify-public&response_type=code in your browser


In [29]:
try:
    response = raw_input("Enter the URL you were redirected to: ")
except NameError:
    response = input("Enter the URL you were redirected to: ")



Enter the URL you were redirected to: http://localhost:8888/callback?code=AQAIwuhTGtRe_s3IhtiG7AKlEU6wFXP9zXfkog1-GNdjJ08O-2oS1VfFjIn0pCduVeqXXjrSz_IrmPLqEjM2RbzcV76-jx5C8EJok6IBZ4PiQxbb0LLvxJK2kc14uA5JqPc9ENwmJltpLYkIcVZtUjdcJrpQVriCgU8DvY_zoLxLYiJj6gBSjb-klaOdAfAd1Glz5doLb2Akk21tvnqEIOp4OjvdhNSimGPJFE2fpmYdyvr9qMX2hFJ3kPhARXpnIiYhf9A


parse code for 2nd step of authentication 

In [30]:
code= response.split("?code=")[1].split("&")[0]

In [31]:
payload = {'redirect_uri': config.SpotifyClient.redirect_uri,
            'code': code,
            'grant_type': 'authorization_code',
            'scope':_normalize_scope(scope)}

response = requests.post(OAUTH_TOKEN_URL, data=payload,
            auth=HTTPBasicAuth(config.SpotifyClient.client_id, config.SpotifyClient.client_secret), verify=True)
if response.status_code is not 200:
    print(response.reason)

token_info = response.json()

['playlist-modify-public', 'playlist-modify-private']


Having access token we can now create a playlist and add tracks to it

In [32]:
access_token=token_info['access_token']

#### Creating empty playlist

In [33]:
CREATE_PLAYLIST_URL='https://api.spotify.com/v1/users/{user_id}/playlists'
ADD_TRACKS_URI='https://api.spotify.com/v1/users/{user_id}/playlists/{playlist_id}/tracks'

In [34]:
import json
payload={"name": "happy "+artist_name}

playlist=requests.post(CREATE_PLAYLIST_URL.format(user_id=config.SpotifyClient.user_id), 
                       headers={"Authorization": "Bearer %s" %token_info['access_token'],
                               "Content-Type":'application/json',
                                "Accept":'application/json'},
                       data = json.dumps(payload))
playlist.raise_for_status()
if playlist.status_code==201:
    print("well done, you just created a new list ",json.dumps(payload))
    playlist_id=playlist.json()["id"]


well done, you just created a new list  {"name": "happy Radiohead"}


#### Adding tracks to new playlist

In [40]:
payload={"uris":",".join(tracks_sorted[:10].uri) }

tracks=requests.post(ADD_TRACKS_URI.format(user_id=config.SpotifyClient.user_id, playlist_id=playlist.json()['id']), 
                       headers={"Authorization": "Bearer %s" %token_info['access_token'],
                               "Content-Type":'application/json',
                                "Accept":'application/json'},
                       params = payload)
if tracks.status_code==201:
    print("It worked! check your spotify and play the list")
    webbrowser.open("https://play.spotify.com/browse/{snapshot_id}".format(snapshot_id=tracks.json()['snapshot_id']))

It worked! check you  spotify and play the list


True